## 1. Setup et Imports

In [1]:
import os
import sys
import torch
from torch.utils.data import DataLoader

# Ajouter le répertoire racine au path
ROOT = os.path.abspath('..')
if ROOT not in sys.path:
    sys.path.append(ROOT)

from src.dpo.models import load_models
from src.dpo.data import PromptDataset, prompt_collate_fn
from src.ppo.ppo_trainer import PPOTrainer
from src.dpo.utils import load_yaml_config

print("✅ Imports réussis")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS available: {torch.backends.mps.is_available()}")

ModuleNotFoundError: No module named 'src'

In [ ]:

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
   

print(f"Device sélectionné: {device}")

## 3. Vérification et Préparation des Données

In [ ]:
# Vérifier si les données de prompts existent
prompts_path = "data/processed/sentiment/prompts.jsonl"

if not os.path.exists(prompts_path):
    print("⚠️  Fichier prompts.jsonl non trouvé!")
    print("📥 Exécution de prepare_prompts.py...")
    !python scripts/prepare_prompts.py
    print("✅ Prompts préparés")
else:
    # Compter le nombre de prompts
    with open(prompts_path, 'r') as f:
        num_prompts = sum(1 for line in f if line.strip())
    print(f"✅ {num_prompts} prompts trouvés dans {prompts_path}")

## 4. Configuration des Hyperparamètres

Vous pouvez modifier ces paramètres selon vos besoins:

In [ ]:
# Charger la config par défaut
config_path = "configs/ppo_sentiment.yaml"
config = load_yaml_config(config_path)

# Afficher les paramètres principaux
print("📊 Configuration PPO:")
print(f"  Model: {config['model']['name']}")
print(f"  Batch size: {config['training']['batch_size']}")
print(f"  Learning rate: {config['training']['learning_rate']}")
print(f"  Epochs: {config['training']['num_epochs']}")
print("\n🎯 Paramètres PPO:")
print(f"  Clip epsilon: {config['ppo']['clip_epsilon']}")
print(f"  Value coef: {config['ppo']['value_coef']}")
print(f"  Entropy coef: {config['ppo']['entropy_coef']}")
print(f"  Target KL: {config['ppo']['target_kl']}")
print(f"  PPO epochs: {config['ppo']['num_ppo_epochs']}")
print("\n🎮 Génération:")
print(f"  Max length: {config['generation']['max_length']}")
print(f"  Temperature: {config['generation']['temperature']}")

### Modifier les paramètres (optionnel)

Décommentez et modifiez si vous voulez changer certains paramètres:

In [ ]:
# Exemple de modifications
# config['training']['batch_size'] = 1  # Réduire si problème de mémoire
# config['training']['num_epochs'] = 2  # Plus d'epochs
# config['ppo']['num_ppo_epochs'] = 2   # Moins d'epochs PPO par batch
# config['generation']['max_length'] = 64  # Réponses plus courtes

print("⚙️  Configuration personnalisée (si modifiée)")

## 5. Chargement des Modèles

In [ ]:
print("📦 Chargement des modèles...")
print("   Cela peut prendre quelques minutes...")

model_name = config["model"]["name"]
dtype = config["model"]["dtype"]

# Charger les modèles (policy et référence)
mb = load_models(model_name, dtype=dtype, device=device)
tokenizer = mb.tokenizer

print(f"✅ Modèles chargés: {model_name}")
print(f"   Policy model: {mb.policy_model.num_parameters():,} paramètres")
print(f"   Device: {mb.device}")

## 6. Préparation du DataLoader

In [ ]:
# Charger le dataset de prompts
prompt_dataset = PromptDataset(config["data"]["prompt_path"])
max_prompt_length = config["data"]["max_prompt_length"]

print(f"📚 Dataset chargé: {len(prompt_dataset)} prompts")

# Fonction de collate
def collate(batch):
    return prompt_collate_fn(
        batch,
        tokenizer=tokenizer,
        max_prompt_length=max_prompt_length,
    )

# DataLoader
prompt_loader = DataLoader(
    prompt_dataset,
    batch_size=config["training"]["batch_size"],
    shuffle=True,
    collate_fn=collate,
)

print(f"✅ DataLoader créé: {len(prompt_loader)} batches")

## 7. Initialisation du PPO Trainer

In [ ]:
print("🏗️  Initialisation du PPO Trainer...")

# Créer le trainer
trainer = PPOTrainer(
    model_bundle=mb,
    prompt_loader=prompt_loader,
    config=config,
)

print("✅ Trainer initialisé")
print(f"   Reward model: {config['reward_model']['name']}")
print(f"   Save dir: {config['logging']['save_dir']}")

## 8. Entraînement PPO 🚀

**⚠️ Attention**: L'entraînement peut prendre du temps, surtout sur CPU!

**Temps estimé**:
- Sur GPU: ~30 min - 1h
- Sur CPU: 2-4h (voire plus selon le hardware)

In [ ]:
import time

print("="*60)
print(f"🚀 Démarrage de l'entraînement PPO")
print(f"   Expérience: {config['experiment_name']}")
print(f"   Device: {device}")
print("="*60)

start_time = time.time()

try:
    # Lancer l'entraînement
    trainer.train()
    
    elapsed = time.time() - start_time
    print("\n" + "="*60)
    print(f"✅ Entraînement terminé!")
    print(f"   Temps total: {elapsed/60:.2f} minutes")
    print("="*60)
    
except KeyboardInterrupt:
    print("\n⚠️  Entraînement interrompu par l'utilisateur")
    print("   Les checkpoints partiels ont été sauvegardés")
    
except Exception as e:
    print(f"\n❌ Erreur pendant l'entraînement: {e}")
    import traceback
    traceback.print_exc()

## 9. Vérification des Checkpoints

In [ ]:
import glob

save_dir = config['logging']['save_dir']
checkpoints = glob.glob(os.path.join(save_dir, "*.pt"))

if checkpoints:
    print(f"✅ {len(checkpoints)} checkpoint(s) sauvegardé(s):")
    for ckpt in sorted(checkpoints):
        size_mb = os.path.getsize(ckpt) / (1024**2)
        print(f"   📁 {os.path.basename(ckpt)} ({size_mb:.1f} MB)")
else:
    print("⚠️  Aucun checkpoint trouvé")

## 10. Test de Génération Rapide

In [ ]:
# Tester la génération avec le modèle entraîné
test_prompts = [
    "The movie was",
    "I think this product is",
    "The customer service was",
]

print("🧪 Test de génération avec le modèle PPO:")
print("="*60)

mb.policy_model.eval()
with torch.no_grad():
    for prompt in test_prompts:
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        outputs = mb.policy_model.generate(
            **inputs,
            max_new_tokens=32,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"\n💬 Prompt: {prompt}")
        print(f"✨ Response: {response}")

print("\n" + "="*60)

## 🎯 Prochaines Étapes

Maintenant que l'entraînement est terminé, vous pouvez:

1. **Évaluer le modèle** avec `eval_sentiment2.py`:
   ```bash
   python scripts/eval_sentiment2.py --method ppo --num_samples 3
   ```

2. **Comparer avec DPO et GRPO**:
   ```bash
   python scripts/compare_methods.py
   ```

3. **Visualiser les résultats**:
   ```bash
   python scripts/visualize_comparison.py
   ```

4. **Modifier les hyperparamètres** et réentraîner pour comparer les performances

## 📊 Notes

- Les checkpoints sont sauvegardés dans `checkpoints/sentiment_ppo/`
- Le modèle PPO inclut un **value head** en plus de la policy
- L'entraînement utilise un **reward model** basé sur le sentiment
- Le **target KL** permet l'early stopping pour la stabilité
- Sur CPU, l'entraînement est plus lent mais plus stable que sur MPS (macOS)